In [10]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [11]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [12]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://flood-api.open-meteo.com/v1/flood"
params = {
	"latitude": 41.0138,
	"longitude": 28.9497,
	"daily": "river_discharge",
	"start_date": "2005-01-01",
	"end_date": "2021-04-07",
	"timezone": "Europe/Moscow"
}
responses = openmeteo.weather_api(url, params=params)

In [13]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 41.0°N 28.949996948242188°E
Elevation nan m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [14]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_river_discharge = daily.Variables(0).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["river_discharge"] = daily_river_discharge

daily_dataframe = pd.DataFrame(data = daily_data)

In [15]:
daily_dataframe

,date,river_discharge
0,2005-01-01 00:00:00+00:00,0.358313
1,2005-01-02 00:00:00+00:00,0.358313
2,2005-01-03 00:00:00+00:00,0.312200
3,2005-01-04 00:00:00+00:00,0.625549
4,2005-01-05 00:00:00+00:00,0.422329
...,...,...
5936,2021-04-03 00:00:00+00:00,0.342765
5937,2021-04-04 00:00:00+00:00,0.655770
5938,2021-04-05 00:00:00+00:00,2.155005
5939,2021-04-06 00:00:00+00:00,1.094112


In [16]:
daily_dataframe.to_csv("../dataset/external/flood.csv", index=False)